In [8]:
import os
from bs4 import BeautifulSoup
import math
import numpy as np
import shapely as shp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.mask

In [9]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [10]:
# Find the coordinate based on the name of the site in our .csv file
df_Site = pd.read_csv(cwd + "//Site.csv")
df_Site.head()

,Number,Site,Serial Number (FLOX),Country,Location,Latitude,Longitude,Priority,Plant functional type (PFT),Reference network,Number of components,Reduce ROI size (600 x 600 m),Move ROI,Comments,PRISMA Availability (900m Buffer ROI Check),PRISMA Earliest Date,PRISMA Latest Date (Before 2024-06-26),Tomi,S2 Cloud
0,1,ATGE,NaN,NaN,NaN,52.466778,12.959778,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,2,2022-05-06,2023-08-14,Y,NaN
1,2,ATLAS-Mohammed V,NaN,Morocco,ATLAS-Mohammed V,33.406152,-5.103319,Low,NaN,NaN,NaN,Yes,Yes,"mixed pixel, complex topography",2,2023-11-23,2023-11-23,NaN,NaN
2,3,AU-MIEMING,JB-042-GW,Austria,Austria,47.316700,10.970300,NaN,NaN,FLOX,NaN,No,No,topography?,0,NaN,NaN,Duplicate,NaN
3,4,BASP,NaN,NaN,NaN,39.049139,-2.075917,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,27,2022-05-09,2024-06-18,Y,NaN
4,5,BE-BRASCHAAT,JB-021-SW,Belgium,Brasschaat,51.307600,4.519900,NaN,NaN,FLOX,NaN,No,No,Check the site with Sentinal 3,7,2020-09-12,2022-06-01,NaN,NaN


In [11]:
for index in range(len(df_Site)):
    temp_Site_Name = df_Site['Site'][index]
    if df_Site.loc[index,"Reduce ROI size (600 x 600 m)"] == "Yes" and df_Site.loc[index,"Move ROI"] == "No":
        print(temp_Site_Name + " has reduced but not moved ROI!")
        temp_GDF = gpd.read_file(cwd_Images_Processed + "\\" + temp_Site_Name + "\\600m.shp")
        src = rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv.tif")
        out_image, out_transform = rio.mask.mask(src, temp_GDF.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})
        with rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv 600m.tif", "w", **out_meta) as dest:
            dest.write(out_image)
    elif df_Site.loc[index,"Reduce ROI size (600 x 600 m)"] == "Yes" and df_Site.loc[index,"Move ROI"] == "Yes":     
        print(temp_Site_Name + " has reduced and moved ROI!")
        temp_GDF = gpd.read_file(cwd_Images_Processed + "\\" + temp_Site_Name + "\\600m Spostamento.shp")
        src = rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv.tif")
        out_image, out_transform = rio.mask.mask(src, temp_GDF.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})
        with rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv 600m Spostamento.tif", "w", **out_meta) as dest:
            dest.write(out_image)

ATLAS-Mohammed V has reduced and moved ROI!
BE-Vie has reduced but not moved ROI!
DE-GEBESE has reduced and moved ROI!
DE-SEL has reduced but not moved ROI!
FR-Bil has reduced and moved ROI!
FR-FBn has reduced and moved ROI!
FR-Pue has reduced and moved ROI!
IS-YATIR has reduced and moved ROI!
IT-BFt has reduced and moved ROI!
IT-SR2 has reduced and moved ROI!
PORT-FOR has reduced and moved ROI!
UK-2 has reduced but not moved ROI!
UK-Amo has reduced and moved ROI!
US-Konza has reduced and moved ROI!
US-LIN has reduced but not moved ROI!
US-OPE has reduced but not moved ROI!
US-SERC has reduced and moved ROI!
Utqiagvik has reduced but not moved ROI!
